In [3]:
import pandas as pd
import datetime as dt
import requests, sys, os
import functools
import json
import math
import statistics
import numpy as np
import pytz

sys.path.append(os.getcwd())

from API_Intergration import irradiance_tags
from datetime import datetime, timedelta, date, time
from pvlib.clearsky import detect_clearsky
from collections import Counter
from pvlib.solarposition import get_solarposition
from pvlib.tracking import singleaxis
from pvlib.irradiance import get_total_irradiance, get_extra_radiation, erbs, disc

In [2]:
plant = "Palmer"
start = datetime(2022,9,1)
end = datetime(2022,9,3)


In [4]:
df = irradiance_tags(plant, start, end)

In [7]:
df

,2096.05.011.MET09.PYR01-IRRADIANCE_POA,2096.05.011.MET10.PYR01-IRRADIANCE_POA,2096.05.011.MET11.PYR01-IRRADIANCE_POA,2096.05.011.MET13.PYR01-IRRADIANCE_POA,2096.05.011.MET17.PYR01-IRRADIANCE_POA,2096.05.011.METREF01.PYR01-IRRADIANCE_POA,2096.05.011.MET12.PYR01-IRRADIANCE_GHI,2096.05.011.MET14.PYR01-IRRADIANCE_GHI,2096.05.011.METREF01.PYR02-IRRADIANCE_GHI,tracking_tilt,tracking_azimuth
Timestamp,,,,,,,,,,,
2022-09-01 00:00:00-04:00,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,180.000000
2022-09-01 00:05:00-04:00,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,180.000000
2022-09-01 00:10:00-04:00,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,0.000000,180.000000
2022-09-01 00:15:00-04:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,180.000000
2022-09-01 00:20:00-04:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,180.000000
2022-09-01 00:25:00-04:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,180.000000
2022-09-01 00:30:00-04:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,180.000000
2022-09-01 00:35:00-04:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,180.000000
2022-09-01 00:40:00-04:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,180.000000


In [6]:
lat, lon = 37.9735, -78.2301

solar_positions = get_solarposition(df.index, lat, lon, 1, method='nrel_numpy')

dni_extra = get_extra_radiation(df.index, epoch_year="2022")


tracking = singleaxis(
    solar_positions['apparent_zenith'], 
    solar_positions['azimuth'], 
    axis_tilt=0,
    axis_azimuth=0,
    max_angle=60,
    gcr=32.0,
    backtrack=False,
)
tracking['surface_tilt'] = tracking['surface_tilt'].fillna(0)
tracking['surface_azimuth'] = tracking['surface_azimuth'].fillna(180)#.replace({xy[0]: xy[1], xy[1]: xy[0]})

tilt = tracking['surface_tilt']
azimuth = tracking['surface_azimuth']


df['tracking_tilt'] = tracking['surface_tilt']
df['tracking_azimuth'] = tracking['surface_azimuth']


In [ ]:
if not functools.reduce(lambda x, y: x and y, [x in df.columns for x in ['dni', 'dhi']]):
    comps = erbs(df['PYR01 - WS02 - GHI 01 - IRRADIANCE_GHI (W/m^2)'], solar_positions['zenith'], df.index)
    dni = disc(df['PYR01 - WS02 - GHI 01 - IRRADIANCE_GHI (W/m^2)'], solar_positions['zenith'], df.index)
    comps['dni'] = dni
    df = pd.concat([df, comps], axis=1)


cs = pd.concat([df, solar_positions], axis=1, join='inner')
    
result = get_total_irradiance(
                                tilt, 
                                azimuth, 
                                cs['zenith'], 
                                cs['azimuth'], 
                                cs['dni'], 
                                cs['PYR01 - WS02 - GHI 01 - IRRADIANCE_GHI (W/m^2)'], 
                                cs['dhi'],
                                dni_extra=dni_extra,
                                model="isotropic"
                            )

cs = pd.concat([cs, result], axis=1)

cs.rename({"poa_global": "poa_Calculated"}, axis=1, inplace=True)


cs[{'poa','PYR02 - WS02 - POA 01 - IRRADIANCE_POA (W/m^2)'}].plot()